In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!pip install jieba
!pip install gensim
!pip install pyLDAvis

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 11.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 10.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 13.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.5/379.5 kB 27.7 MB/s 

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import re
import jieba
from pprint import pprint

import os
import sys
sys.stderr = open(os.devnull, "w")  # silence stderr
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
sys.stderr = sys.__stderr__  # unsilence stderr

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # 3.3.x 版本之前请使用 import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from '

In [3]:
df = pd.read_excel(r'新闻 - 副本.xlsx') 

# 去除文本中的表情字符（只保留中文）
def clear_character(sentence):    
    pattern = re.compile('[^\u4e00-\u9fa5]')   
    line = re.sub(pattern, '', sentence)   
    new_sentence = ''.join(line.split())
    return new_sentence

df["content"]=df["content"].astype(str)

train_text = [clear_character(data) for data in df["content"]]
train_seg_text = [jieba.lcut(s) for s in train_text]


# 加载停用词
stop_words_path = "stopwords.txt"

def get_stop_words():
    return set([item.strip() for item in open(stop_words_path, 'r').readlines()])

stopwords = get_stop_words()


In [4]:
# 去掉文本中的停用词
def drop_stopwords(line):
    line_clean = []
    for word in line:
        if word in stopwords:
            continue
        line_clean.append(word)
    return line_clean

In [5]:
train_st_text = [drop_stopwords(s) for s in train_seg_text]

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(train_st_text, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[train_st_text], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [8]:
data_words_bigrams = make_bigrams(train_st_text)
print(data_words_bigrams[:1])

[['新闻_发布会', '现场', '人民网', '王慧摄', '人民网_呼和浩特', '日电', '王慧', '上午', '内蒙古', '全区', '公安机关', '贯彻落实', '全国公安', '工作', '会议', '精神', '三周年', '成效', '新闻_发布会', '呼和浩特', '据介绍', '全区', '公安机关', '以大_数据', '牵引', '全力', '推进', '北疆', '智慧', '公安', '建设', '实施', '科技_兴警', '战略', '数据', '公安', '工作', '创新', '发展', '引擎', '培育', '战斗力', '生成_增长点', '助力', '公安机关', '打击犯罪', '提升', '服务', '社会', '能力', '水平', '提高', '群众', '办事效率', '服务', '群众', '优化', '升级', '全区', '互联网', '公安', '政务', '服务平台', '手机', '微信_公众', '自治区', '一体化', '在线', '政务', '服务', '公安部', '盟市', '公安', '政务', '服务', '平台', '互联互通', '内蒙古自治区', '公安', '政务', '服务', '同蒙', '速办', '对接', '深化', '政务', '服务', '数据', '共享', '公安', '政务', '服务', '事项', '材料_时限', '更好', '支撑', '一网通', '网上_掌上', '群众', '跑腿', '数据', '公安', '数据', '政府部门', '支撑', '截至', '依托', '自治区', '政务信息_共享', '平台', '自治区', '各盟', '政务', '部门', '提供', '全区', '常住人口', '查询', '服务', '近万', '次年', '政务', '部门', '需求', '遵循', '规范', '前提', '批量', '核查', '方式', '自治区', '社厅', '民政厅', '教育厅', '部门', '提供', '核查', '服务', '万余条', '助力', '自治区', '相关', '部门', '人口', '核验', '疫情_防控', '全民', '参保', '全区', '低保', '发放', '高考',

In [9]:
id2word = corpora.Dictionary(data_words_bigrams)     # Create Dictionary
id2word.save_as_text("dictionary")                   # save dict
texts = data_words_bigrams                           # Create Corpus
corpus = [id2word.doc2bow(text) for text in texts]   # Term Document Frequency
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 1), (20, 1), (21, 7), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 2), (62, 1), (63, 2), (64, 8), (65, 1), (66, 1), (67, 1), (68, 4), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 10), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 4), (100, 1), (101, 5), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 4), (109, 1), (110, 1

In [10]:
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])


[[('一体化', 1), ('一网通', 1), ('万余条', 1), ('三周年', 1), ('上午', 1), ('事项', 1), ('互联互通', 1), ('互联网', 1), ('人口', 1), ('人民网', 1), ('人民网_呼和浩特', 1), ('以大_数据', 1), ('优化', 1), ('会议', 1), ('低保', 1), ('依托', 1), ('信息', 1), ('全力', 1), ('全区', 5), ('全国公安', 1), ('全民', 1), ('公安', 7), ('公安机关', 3), ('公安部', 1), ('共享', 1), ('内蒙古', 1), ('内蒙古自治区', 1), ('分享', 1), ('创新', 1), ('前提', 1), ('办事效率', 1), ('助力', 2), ('北疆', 1), ('升级', 1), ('参保', 1), ('发展', 1), ('发放', 1), ('各盟', 1), ('同蒙', 1), ('呼和浩特', 1), ('在线', 1), ('培育', 1), ('实施', 1), ('对接', 1), ('工作', 2), ('常住人口', 1), ('平台', 2), ('建设', 1), ('引擎', 1), ('微信_公众', 1), ('成效', 1), ('战斗力', 1), ('战略', 1), ('截至', 1), ('手机', 1), ('打击犯罪', 1), ('批量', 1), ('据介绍', 1), ('推进', 1), ('提供', 2), ('提供数据', 1), ('提升', 2), ('提高', 1), ('支撑', 2), ('政务', 8), ('政务信息_共享', 1), ('政府部门', 1), ('教育厅', 1), ('数据', 4), ('新闻_发布会', 2), ('方式', 1), ('日电', 1), ('智慧', 1), ('更好', 1), ('服务', 10), ('服务平台', 1), ('材料_时限', 1), ('查询', 1), ('核准', 1), ('核查', 2), ('核验', 1), ('次年', 1), ('民政厅', 1), ('水平', 1), ('深化', 1), ('

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.015*"交通" + 0.013*"车辆" + 0.011*"交警" + 0.008*"停车" + 0.006*"出行" + 0.006*"人员" '
  '+ 0.006*"市民" + 0.006*"现场" + 0.005*"情况" + 0.005*"违法"'),
 (1,
  '0.045*"公安" + 0.036*"工作" + 0.022*"教育" + 0.015*"发展" + 0.014*"建设" + 0.012*"人民" '
  '+ 0.010*"队伍" + 0.009*"学习" + 0.009*"教育_整顿" + 0.009*"会议"'),
 (2,
  '0.086*"智慧" + 0.019*"建设" + 0.017*"平台" + 0.014*"服务" + 0.012*"管理" + 0.011*"数据" '
  '+ 0.011*"城市" + 0.009*"系统" + 0.008*"打造" + 0.007*"智能"'),
 (3,
  '0.017*"企业" + 0.014*"农业" + 0.014*"发展" + 0.013*"生产" + 0.012*"公司" + 0.012*"产业" '
  '+ 0.011*"税务" + 0.010*"项目" + 0.007*"办税" + 0.006*"乡村_振兴"'),
 (4,
  '0.018*"治理" + 0.017*"警务" + 0.013*"执法" + 0.012*"工作" + 0.012*"社区" + 0.011*"平安" '
  '+ 0.011*"公安" + 0.011*"机制" + 0.010*"基层" + 0.010*"建设"'),
 (5,
  '0.028*"民警" + 0.019*"群众" + 0.017*"诈骗" + 0.017*"公安局" + 0.016*"犯罪_嫌疑人" + '
  '0.014*"派出所" + 0.010*"抓获" + 0.009*"宣传" + 0.008*"案件" + 0.008*"分局"'),
 (6,
  '0.024*"训练" + 0.022*"练兵" + 0.014*"实战" + 0.009*"全警_实战" + 0.009*"教官" + '
  '0.006*"考核" + 0.005*"培训" + 0.005*"最小_作战" + 0

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.)


Perplexity:  -9.833544540548708


In [17]:
# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)   # 越高越好

In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)   # 根据版本信息选择
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.120781 -0.285391       1        1  29.113480
9     -0.224086  0.096763       2        1  17.459686
5     -0.131704  0.179219       3        1  11.361564
0     -0.088073  0.026518       4        1  10.092825
1     -0.015441  0.038331       5        1   9.345284
3      0.185619 -0.167606       6        1   6.953499
4     -0.065189 -0.073672       7        1   6.547981
8      0.063530 -0.056031       8        1   5.559962
7      0.162236  0.180839       9        1   3.335792
6      0.233888  0.061030      10        1   0.229927, topic_info=     Term          Freq         Total Category  logprob  loglift
72     智慧  48505.000000  48505.000000  Default  30.0000  30.0000
74     服务  14944.000000  14944.000000  Default  29.0000  29.0000
21     公安  25452.000000  25452.000000  Default  28.0000  28.0000
44     工作  20069.000000  20069.000000  Default  27.0000  27.0000
237    民警   8962.000000   8962.000000  Default  26.0000  26.0000
...   ...           ...           ...      ...      ...      ...
518    培训     23.942303   1034.896858  Topic10  -5.2237   2.3088
2529   素质     11.745379    141.121125  Topic10  -5.9359   3.5890
5457   技能     13.442804    288.009638  Topic10  -5.8009   3.0106
83     水平     15.223117   1685.377915  Topic10  -5.6765   1.3682
177    处置     10.619154   1626.422088  Topic10  -6.0367   1.0437

[604 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
6748       4  0.994173    一辆
3277       3  0.674671    万元
3277       6  0.324013    万元
21438      6  0.995197    三亚
5148       6  0.996258    三农
...      ...       ...   ...
2504       1  0.077772    高速
2504       4  0.919265    高速
12612      4  0.995892  高速公路
20725      7  0.990615    鸟类
7862       7  0.987191    黄河

[1092 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 6, 1, 2, 4, 5, 9, 8, 7])

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 